In [1]:
import os, sys, rdflib
import numpy as np, pandas as pd
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
import rdflib.plugins.sparql as sparql
from SPARQLWrapper import SPARQLWrapper, JSON
sys.path.append(os.path.abspath("./util"))
from misc import *

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn import metrics
from hdbscan import HDBSCAN
from sklearn.cluster import DBSCAN, KMeans, SpectralClustering, AgglomerativeClustering

In [2]:
%%time
%store -r uri_type
if "uri_type" not in locals():
    dbpedia_type_graph = rdflib.Graph()
    dbpedia_type_graph.parse('./dataset/instance_types_en_uris_it.nt', format="nt" )
    dbpedia_type_graph.parse('./dataset/instance_types_en.nt', format="nt" )

CPU times: user 2.98 s, sys: 875 ms, total: 3.85 s
Wall time: 6.44 s


In [3]:
walks_path = "./dataset/embedding_5_7_dbpedia_3.9_infobox_properties_no_literal.nt.txt"

vocab_sequences = get_sequences(walks_path)
train_sequences = get_sequences(walks_path)

model = Word2Vec(min_count=1, window=5, negative=5, size=100)

model.build_vocab(vocab_sequences)
model.train(train_sequences)

9023075

In [4]:
%store -r uri_type
if "uri_type" not in locals():
    uri_type = dict()
    for uri in model.vocab:
        types = {row[0].n3() for row in get_types(uri, dbpedia_type_graph)}
        if len(types) > 0:
            uri_type[uri] = types
    %store uri_type

In [5]:
embeds = np.asarray([model[uri] for uri in uri_type])
uris   = np.asarray([uri for uri in uri_type])
# labels = np.asarray([1 if is_person(mid, mid2types) else 0 for mid in mid2types])

# tsne = TSNE(n_components=2)
# %store -r twodims
# if "twodims" not in locals():
#     twodims = tsne.fit_transform(embeds)
#     %store twodims

ground_truth = get_ground_truth(uris, uri_type)

# colors  = [get_color(i) for i in ground_truth]

In [ ]:
%%time
# hdbscan = HDBSCAN(min_cluster_size=30)
# dbscan  = DBSCAN(eps=0.4, min_samples=5)
kmeans  = KMeans(n_clusters=10)
# spect   = SpectralClustering(n_clusters=7, eigen_solver='arpack', affinity="nearest_neighbors")
# aggl    = AgglomerativeClustering(linkage="average", affinity="cityblock", n_clusters=7)

# hdbscan_labels = hdbscan.fit_predict(embeds.astype(np.double))
# dbscan_labels  = dbscan.fit_predict(embeds.astype(np.double))
kmeans_labels  = kmeans.fit_predict(embeds.astype(np.double))

# %store -r spect_labels_dbpedia
# %store -r aggl_labels_dbpedia
# 
# if "spect_labels_dbpedia" not in locals():
#     spect_labels_dbpedia = spect.fit_predict(embeds.astype(np.double))
#     aggl_labels_dbpedia  = aggl.fit_predict(embeds.astype(np.double))
#     %store spect_labels_dbpedia
#     %store aggl_labels_dbpedia

# cluster_colors = [get_color(clust) for clust in spect_labels]

CPU times: user 3min 16s, sys: 7.59 s, total: 3min 23s
Wall time: 1min 8s


In [ ]:
metrics_class_df = pd.concat([
    # clust_metrics(ground_truth, hdbscan_labels, embeds, metric="euclidean", name="HDBSCAN"),
    # clust_metrics(ground_truth, dbscan_labels, embeds, metric="euclidean", name="DBSCAN"),
    clust_metrics(ground_truth, kmeans_labels, embeds, metric="euclidean", name="KMeans"),
    # clust_metrics(ground_truth, spect_labels_dbpedia, embeds, metric="euclidean", name="Spectral"),
    # clust_metrics(ground_truth, aggl_labels_dbpedia, embeds, metric="euclidean", name="Agglomerative")
])

metrics_class_df# 